# 🤖 Ollama Chat Interface

Interactive chat interface for your local Ollama Mistral model.

## 📦 Setup & Dependencies

Run this cell first to install required packages:

In [ ]:
# Install dependencies (run once)
# !pip install ipywidgets requests --quiet

## 🔧 Configuration

In [ ]:
import requests
import json
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox, HBox, Text, HTML as HTMLWidget, Textarea, Output

# Configuration
OLLAMA_URL = "http://127.0.0.1:11434"
MODEL_NAME = "mistral:7b"
# MODEL_NAME = "gemma3:1b-it-qat"

# Chat history
chat_history = []

print("✅ Configuration loaded")
print(f"Model: {MODEL_NAME}")
print(f"Endpoint: {OLLAMA_URL}")

## 🔍 Test Connection

In [ ]:
def check_connection():
    """Test connection to Ollama."""
    try:
        response = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
        if response.status_code == 200:
            print("✅ Successfully connected to Ollama!")
            models = response.json().get('models', [])
            print(f"\nAvailable models: {len(models)}")
            for model in models:
                print(f"  - {model['name']}")
            return True
        else:
            print(f"❌ Connection failed: Status {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama")
        print("\nMake sure:")
        print("  1. Docker container is running: docker ps")
        print("  2. Port 11434 is mapped correctly")
        return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

check_connection()

## 💬 Chat Functions

In [ ]:
def send_message_to_ollama(prompt: str):
    """Send message to Ollama and get response."""
    try:
        response = requests.post(
            f"{OLLAMA_URL}/api/generate",
            json={
                "model": MODEL_NAME,
                "prompt": prompt,
                "stream": False
            },
            timeout=120
        )
        
        if response.status_code == 200:
            return response.json()['response']
        else:
            return f"Error: Status code {response.status_code}"
    
    except requests.exceptions.ConnectionError:
        return "❌ Error: Cannot connect to Ollama. Is the container running?"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def format_chat_message(role: str, message: str, timestamp: str = None):
    """Format a chat message as HTML."""
    if timestamp is None:
        timestamp = datetime.now().strftime("%H:%M:%S")
    
    if role == "user":
        bg_color = "#2b5278"
        icon = "👤"
        align = "right"
    else:
        bg_color = "#1e3a5f"
        icon = "🤖"
        align = "left"
    
    return f"""
    <div style="margin: 10px 0; text-align: {align};">
        <div style="display: inline-block; max-width: 70%; background-color: {bg_color}; 
                    padding: 12px; border-radius: 10px; text-align: left;">
            <div style="font-size: 0.8em; color: #aaa; margin-bottom: 5px;">
                {icon} <strong>{role.upper()}</strong> <span style="float: right;">{timestamp}</span>
            </div>
            <div style="color: #fff; white-space: pre-wrap; word-wrap: break-word;">{message}</div>
        </div>
    </div>
    """

print("✅ Chat functions loaded")

# Test Response Manually without Chat Interface

In [ ]:
def test_llm(prompt: str):
    """Send message to LLM and get a stream of live processed text."""
    try:
        response = requests.post(
            f"{OLLAMA_URL}/api/generate",
            json={
                "model": MODEL_NAME,
                "prompt": prompt,
                "stream": False
            },
            timeout=120,
            stream=True
        )
        
        if response.status_code == 200:
            print("Response received successfully.")
            for line in response.iter_lines(decode_unicode=True):
                if line:
                    try:
                        json_data = json.loads(line)
                        if 'response' in json_data:
                            print(json_data['response'], end='')
                    except json.JSONDecodeError:
                        print("Error decoding JSON:", line)
        else:
            print(f"Failed to get response: {response.status_code} - {response.text}")
    
    except requests.exceptions.ConnectionError:
        return "❌ Error: Cannot connect to Ollama. Is the container running?"
    except Exception as e:
        return f"❌ Error: {str(e)}"
    
test_llm('Introduce yourself. Be sarcastic.')

## 🎨 Interactive Chat Interface

Run the cell below to start chatting!

In [ ]:
# Create widgets
chat_output = Output(layout=Layout(height='400px', overflow_y='auto', border='1px solid #444', 
                                    padding='10px', background_color='#0e1117'))
input_box = Textarea(
    placeholder='Type your message here...',
    layout=Layout(width='70%', height='60px')
)
send_button = Button(
    description='Send',
    button_style='primary',
    layout=Layout(width='15%', height='60px')
)
clear_button = Button(
    description='Clear Chat',
    button_style='warning',
    layout=Layout(width='15%', height='60px')
)
status_label = HTMLWidget(
    value='<p style="color: #888;">Ready to chat! Type a message and click Send.</p>'
)

def display_chat_history():
    """Display all chat messages."""
    with chat_output:
        clear_output()
        for msg in chat_history:
            display(HTML(format_chat_message(msg['role'], msg['content'], msg['timestamp'])))

def on_send_click(b):
    """Handle send button click."""
    user_message = input_box.value.strip()
    
    if not user_message:
        status_label.value = '<p style="color: orange;">⚠️ Please enter a message</p>'
        return
    
    # Clear input
    input_box.value = ''
    
    # Add user message to history
    timestamp = datetime.now().strftime("%H:%M:%S")
    chat_history.append({
        'role': 'user',
        'content': user_message,
        'timestamp': timestamp
    })
    
    # Display updated chat
    display_chat_history()
    
    # Show loading status
    status_label.value = '<p style="color: #4CAF50;">🔄 Thinking...</p>'
    
    # Get response from Ollama
    response = send_message_to_ollama(user_message)
    
    # Add assistant response to history
    timestamp = datetime.now().strftime("%H:%M:%S")
    chat_history.append({
        'role': 'assistant',
        'content': response,
        'timestamp': timestamp
    })
    
    # Display updated chat
    display_chat_history()
    
    # Update status
    if response.startswith('❌'):
        status_label.value = '<p style="color: red;">Error occurred. Check connection.</p>'
    else:
        status_label.value = f'<p style="color: #4CAF50;">✅ Response received | Messages: {len(chat_history)}</p>'

def on_clear_click(b):
    """Handle clear button click."""
    global chat_history
    chat_history = []
    with chat_output:
        clear_output()
    status_label.value = '<p style="color: #888;">Chat cleared. Ready for new conversation.</p>'

# Attach event handlers
send_button.on_click(on_send_click)
clear_button.on_click(on_clear_click)

# Allow Enter key to send (Shift+Enter for new line)
def handle_input(change):
    if '\n' in change['new'] and not change['new'].endswith('\n\n'):
        # Simple enter pressed
        input_box.value = change['new'].replace('\n', '')
        on_send_click(None)

# Layout
input_row = HBox([input_box, send_button, clear_button], layout=Layout(width='100%'))
chat_interface = VBox([
    HTMLWidget(value='<h3 style="color: #4CAF50;">💬 Chat with Mistral 7B</h3>'),
    status_label,
    chat_output,
    HTMLWidget(value='<p style="color: #888; font-size: 0.9em; margin-top: 10px;">'
                     'Tip: Type your message and click Send or press Enter</p>'),
    input_row
], layout=Layout(width='100%', padding='20px', background_color='#1a1a1a', border_radius='10px'))

# Display the interface
display(chat_interface)

print("\n🎉 Chat interface ready! Start typing below.")

## 📊 Chat Statistics

In [ ]:
# View chat statistics
print(f"Total Messages: {len(chat_history)}")
print(f"User Messages: {len([m for m in chat_history if m['role'] == 'user'])}")
print(f"Assistant Messages: {len([m for m in chat_history if m['role'] == 'assistant'])}")

if chat_history:
    print(f"\nFirst message: {chat_history[0]['timestamp']}")
    print(f"Last message: {chat_history[-1]['timestamp']}")